# Лабораторная работа №7
## Классификация обзоров фильмов
## Группа: БФИ1901

### Цель работы:
Используя датасет IMDb, провести обучение рекуррентной нейронной сети.

### Задание:
* Ознакомиться с рекуррентными нейронными сетями
* Изучить способы классификации текста
* Ознакомиться с ансамблированием сетей
* Построить ансамбль сетей, который позволит получать точность не менее 97%

### Ход работы:
#### Импорт зависимостей и получение данных
Начнем с импорта необходимых зависимостей для предварительной обработки данных и построения модели.

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Загрузим датесет IMDb, который уже встроен в Keras. Поскольку мы не хотим иметь данные обучения и тестирования в пропорции 50/50, мы сразу же объединим эти данные после загрузки для последующего разделения в пропорции 80/20:

In [2]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

#### Обработка полученных данных
Нам нужно обрезать и дополнить входные последовательности так, чтобы они были одинаковой длины для моделирования:

In [3]:
top_words = 10000
X_test = data[:10000]
y_test = targets[:10000]
X_train = data[10000:]
y_train = targets[10000:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

#### Настройка и обучение модели
Создадим рекуррентную сверточную нейронную сеть, скомпилируем её и запустим процесс обучения. 

In [4]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
625/625 [==============================] - 246s 392ms/step - loss: 0.4002 - accuracy: 0.8185 - val_loss: 0.2975 - val_accuracy: 0.8758
Epoch 2/3
625/625 [==============================] - 260s 416ms/step - loss: 0.2453 - accuracy: 0.9053 - val_loss: 0.2853 - val_accuracy: 0.8809
Epoch 3/3
625/625 [

#### Вопрос 1: Найти набор оптимальных ИНС для классификации текста
Ниже представлен код оригинальной программы. Запустим его, чтобы посмотреть результат обучения.

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb

top_words = 10000
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

X_test = data[:10000]
y_test = targets[:10000]
X_train = data[10000:]
y_train = targets[10000:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           320000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
625/625 [==============================] - 286s 456ms/step - loss: 0.4510 - accuracy: 0.7849 - val_loss: 0.3273 - val_accuracy: 0.8657
Epoch 2/3
625/625 [==============================] - 282s 451ms/step - loss: 0.2664 - accuracy: 0.8942 - val_loss: 0.2968 - val_accuracy: 0.8775
Epoch 3/3
625/625

Добавим в нашу модель слой свертки и подвыборки после слоя Embedding для уменьшения времени обучения без вреда для эффективности обучения.

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb

top_words = 10000
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

X_test = data[:10000]
y_test = targets[:10000]
X_train = data[10000:]
y_train = targets[10000:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 32)           320000    
                                                                 
 conv1d (Conv1D)             (None, 500, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 32)          0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 100)               53200     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 376,405
Trainable params: 376,405
Non-trainable params: 0
________________________________________________

Добавим в нашу модель несколько слоев Dropout:

In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb

top_words = 10000
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

X_test = data[:10000]
y_test = targets[:10000]
X_train = data[10000:]
y_train = targets[10000:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(LSTM(100))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 32)           320000    
                                                                 
 dropout (Dropout)           (None, 500, 32)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 32)           3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 250, 32)          0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 250, 32)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                      

#### Вопрос 2: Провести ансамблирование моделей

Проведем ансамблирование моделей. Запустим обучение 3-х моделей, протестируем их и усредним результаты тестирования.

In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import statistics
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb

def create_model(top_words, embedding_vecor_length, max_review_length):
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
    model.add(Dropout(0.2, noise_shape=None, seed=None))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2, noise_shape=None, seed=None))
    model.add(LSTM(100))
    model.add(Dropout(0.2, noise_shape=None, seed=None))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

top_words = 10000
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

X_test = data[:10000]
y_test = targets[:10000]
X_train = data[10000:]
y_train = targets[10000:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32
model_num = 3

model_list = list()

for i in range (model_num):
    model = create_model(top_words, embedding_vecor_length, max_review_length)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
    model_list.append(model)
    
yhats = list()

for model in model_list:
    yhats.append(model.predict(X_test))
    
newlist = list()
meanlist = list()
finallist = list()

for i in range(len(yhats)):
    newlist.append(yhats[i].flatten())
    
for i in range(len(newlist[0])):
    for j in range(len(newlist)):
        meanlist.append(newlist[j][i])
    finallist.append(statistics.mean(meanlist))
    meanlist.clear()
    
print("Ensemble prediction:\n")
print(finallist)

Epoch 1/3
625/625 [==============================] - 134s 212ms/step - loss: 0.3877 - accuracy: 0.8127 - val_loss: 0.3025 - val_accuracy: 0.8785
Epoch 2/3
625/625 [==============================] - 138s 220ms/step - loss: 0.2226 - accuracy: 0.9144 - val_loss: 0.2779 - val_accuracy: 0.8856
Epoch 3/3
625/625 [==============================] - 128s 205ms/step - loss: 0.1782 - accuracy: 0.9332 - val_loss: 0.2537 - val_accuracy: 0.8978
Epoch 1/3
625/625 [==============================] - 136s 215ms/step - loss: 0.3864 - accuracy: 0.8162 - val_loss: 0.2954 - val_accuracy: 0.8838
Epoch 2/3
625/625 [==============================] - 7944s 13s/step - loss: 0.2299 - accuracy: 0.9121 - val_loss: 0.2613 - val_accuracy: 0.8995
Epoch 3/3
625/625 [==============================] - 255s 408ms/step - loss: 0.1883 - accuracy: 0.9286 - val_loss: 0.2924 - val_accuracy: 0.8913
Epoch 1/3
625/625 [==============================] - 231s 365ms/step - loss: 0.3858 - accuracy: 0.8219 - val_loss: 0.2715 - val_acc

#### Вопрос 3: Написать функцию/функции, которые позволят загружать текст и получать результат ансамбля сетей

Разделим нашу программу на блоки, которые позволили бы быстрее работать с пользовательским текстом:

In [9]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
from keras.datasets import imdb

def input_user_text():
    user_text = input().lower().split()

    index = imdb.get_word_index()
    reverse_index = dict([(key, value) for (key, value) in index.items()])

    conv_text_arr = []

    for word in user_text:
        conv_text_arr.append(reverse_index.get(word, 0))

    result = np.array([conv_text_arr])
    return result

In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import statistics
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb

def create_model(top_words, embedding_vecor_length, max_review_length):
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
    model.add(Dropout(0.1, noise_shape=None, seed=None))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.1, noise_shape=None, seed=None))
    model.add(LSTM(100))
    model.add(Dropout(0.1, noise_shape=None, seed=None))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

top_words = 10000
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

X_test = data[:250]
y_test = targets[:250]
X_train = data[1:]
y_train = targets[1:]


max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32
model_num = 3

model_list = list()

for i in range (model_num):
    model = create_model(top_words, embedding_vecor_length, max_review_length)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
    model_list.append(model)

Epoch 1/3
782/782 [==============================] - 292s 372ms/step - loss: 0.3575 - accuracy: 0.8327 - val_loss: 0.1525 - val_accuracy: 0.9440
Epoch 2/3
782/782 [==============================] - 205s 262ms/step - loss: 0.2130 - accuracy: 0.9173 - val_loss: 0.1151 - val_accuracy: 0.9680
Epoch 3/3
782/782 [==============================] - 269s 344ms/step - loss: 0.1746 - accuracy: 0.9357 - val_loss: 0.0742 - val_accuracy: 0.9840
Epoch 1/3
782/782 [==============================] - 284s 361ms/step - loss: 0.3539 - accuracy: 0.8383 - val_loss: 0.1806 - val_accuracy: 0.9320
Epoch 2/3
782/782 [==============================] - 296s 378ms/step - loss: 0.2272 - accuracy: 0.9115 - val_loss: 0.1582 - val_accuracy: 0.9520
Epoch 3/3
782/782 [==============================] - 237s 304ms/step - loss: 0.1751 - accuracy: 0.9350 - val_loss: 0.1894 - val_accuracy: 0.9240
Epoch 1/3
782/782 [==============================] - 253s 322ms/step - loss: 0.3654 - accuracy: 0.8313 - val_loss: 0.1598 - val_ac

In [29]:
user_text = input_user_text()
user_text = sequence.pad_sequences(user_text, maxlen=max_review_length)

yhats = list()

for model in model_list:
    yhats.append(model.predict(user_text))
    
newlist = list()
meanlist = list()
finallist = list()

for i in range(len(yhats)):
    newlist.append(yhats[i].flatten())
    
for i in range(len(newlist[0])):
    for j in range(len(newlist)):
        meanlist.append(newlist[j][i])
    finallist.append(statistics.mean(meanlist))
    meanlist.clear()
    
print("Ensemble prediction:")
print(finallist)

if finallist[0] >= 0.5:
    print("Positive")
else:
    print("Negative")

nice film
Ensemble prediction:
[0.54787654]
Positive


#### Вывод

Мы произвели обучение рекуррентной нейронной сети по датасету IMDB. Кроме того, мы создали ансамбль моделей и протестировали его на пользовательском тексте.